# Training on multiple topologies
Eamon Whalen

In [1]:
import sys
import glob
import numpy as np
import pandas as pd
import altair as alt
from sklearn.model_selection import train_test_split

sys.path.append('./models')
from feastnetSurrogateModel import FeaStNet

sys.path.append('./readers')
from loadGhGraphs import loadGhGraphs

sys.path.append('./visualization')
from altTrussViz import plotTruss, interactiveErrorPlot

sys.path.append('./util')
from gcnSurrogateUtil import *

## 1. Load simulation data

In [2]:
trainSets ,valSets ,testSets = {}, {}, {}

doeFiles = np.sort(glob.glob("/home/ewhalen/projects/data/trusses/2D_Truss_v1.3/*.csv"))
for doeFile in doeFiles[:3]:
    designName = doeFile.split('/')[-1].split('_N')[0]
    print(f'loading {designName}')
    allGraphsUnfiltered = loadGhGraphs(doeFile, NUM_DV=5)
    allGraphs = filterbyDisp(allGraphsUnfiltered, 0.9)
    trainData, valData, testData = partitionGraphList(allGraphs)
    trainSets[designName] = trainData
    valSets[designName] = valData
    testSets[designName] = testData

loading design_5
loading design_6
loading design_7


## 2. Train on each group seperately

In [3]:
resultsList = []
saveDir = './results/topoTest01/'
epochs = 2

In [4]:
for trainName, trainSet in trainSets.items():
    print('training on '+trainName)

    # train
    gcn = FeaStNet()
    history = gcn.trainModel(trainSet, valSets[trainName], epochs=epochs, batch_size=256, flatten=True, logTrans=False, 
                             ssTrans=True, saveDir=saveDir+trainName)

    display(plotHistory(history))

    # test
    print('testing on '+trainName+'\n')
    resultsDict = gcn.testModel(testSets[trainName], level='field')
    resultsDict['Trained on'] = ['test group']*len(resultsDict['mse'])
    resultsDict['Tested on'] = [trainName]*len(resultsDict['mse'])
    results = pivotDict(resultsDict)
    resultsList.extend(results)
        
pd.DataFrame(resultsList)

training on design_5
epoch: 0   trainLoss: 9.9687e-01   valLoss:1.0413e+00  time: 1.28e+00
epoch: 1   trainLoss: 8.9824e-01   valLoss:1.0386e+00  time: 9.82e-01
loading checkpoint 1


alt.Chart(...)

testing on design_5

training on design_6
epoch: 0   trainLoss: 9.5163e-01   valLoss:1.1936e+00  time: 1.01e+00
epoch: 1   trainLoss: 8.7033e-01   valLoss:1.1914e+00  time: 9.71e-01
loading checkpoint 1


alt.Chart(...)

testing on design_6

training on design_7
epoch: 0   trainLoss: 9.6183e-01   valLoss:9.5726e-01  time: 1.01e+00
epoch: 1   trainLoss: 8.3152e-01   valLoss:9.5431e-01  time: 1.03e+00
loading checkpoint 1


alt.Chart(...)

testing on design_7



,mse,mae,mre,maxAE,mae/peak,maxAE/peak,relEAtPeak,Trained on,Tested on
0,0.000163,0.011892,0.460831,0.019827,0.725857,1.210207,0.033278,test group,design_5
1,0.000198,0.013116,0.483197,0.023848,0.393213,0.714965,0.525229,test group,design_5
2,0.000288,0.013556,0.378542,0.041126,0.238307,0.722956,0.721755,test group,design_5
3,0.001289,0.031469,0.624846,0.081098,0.324650,0.836656,0.836643,test group,design_5
4,0.000174,0.012435,0.481871,0.019110,0.876238,1.346615,0.116389,test group,design_5
...,...,...,...,...,...,...,...,...,...
535,0.000278,0.014035,0.561400,0.036675,0.442064,1.155193,0.818686,test group,design_7
536,0.000022,0.003511,0.195175,0.011931,0.198727,0.675327,0.674371,test group,design_7
537,0.000046,0.005212,0.248897,0.015140,0.259772,0.754527,0.712952,test group,design_7
538,0.000028,0.004246,0.210525,0.012490,0.232949,0.685295,0.684250,test group,design_7


In [5]:
gcn.checkptFile

'./results/topoTest01/design_7/checkpoint_1'

In [6]:
gcn.ssTrans

True

In [7]:
import torch
gcn2 = torch.load(gcn.checkptFile)

In [8]:
gcn2.checkptFile

'./results/topoTest01/design_7/checkpoint_1'

## 3. Train on all groups at once

In [9]:
allTrainData, allValData = [], []
print('training on all groups')
for name, data in trainSets.items():
    allTrainData = allTrainData + data
    allValData = allValData + valSets[name]

gcn = FeaStNet()
history = gcn.trainModel(allTrainData, allValData, epochs=epochs, batch_size=256, flatten=True, logTrans=False, 
                         ssTrans=True, saveDir=saveDir+'allGroups')

display(plotHistory(history))

# test
for testName, testSet in testSets.items():
    print('testing on '+testName+'\n')
    resultsDict = gcn.testModel(testSet, level='field')
    resultsDict['Trained on'] = ['all groups']*len(resultsDict['mse'])
    resultsDict['Tested on'] = [testName]*len(resultsDict['mse'])
    results = pivotDict(resultsDict)
    resultsList.extend(results)
    
pd.DataFrame(resultsList)

training on all groups
epoch: 0   trainLoss: 9.0992e-01   valLoss:1.0895e+00  time: 3.21e+00
epoch: 1   trainLoss: 8.1494e-01   valLoss:1.1181e+00  time: 3.27e+00
loading checkpoint 0


alt.Chart(...)

testing on design_5

testing on design_6

testing on design_7



,mse,mae,mre,maxAE,mae/peak,maxAE/peak,relEAtPeak,Trained on,Tested on
0,0.000163,0.011892,0.460831,0.019827,0.725857,1.210207,0.033278,test group,design_5
1,0.000198,0.013116,0.483197,0.023848,0.393213,0.714965,0.525229,test group,design_5
2,0.000288,0.013556,0.378542,0.041126,0.238307,0.722956,0.721755,test group,design_5
3,0.001289,0.031469,0.624846,0.081098,0.324650,0.836656,0.836643,test group,design_5
4,0.000174,0.012435,0.481871,0.019110,0.876238,1.346615,0.116389,test group,design_5
...,...,...,...,...,...,...,...,...,...
1075,0.000499,0.019292,0.703043,0.044364,0.607648,1.397381,0.421762,all groups,design_7
1076,0.000096,0.009059,0.381865,0.017203,0.512769,0.973743,0.055565,all groups,design_7
1077,0.000086,0.007491,0.307010,0.019104,0.373341,0.952111,0.144820,all groups,design_7
1078,0.000061,0.006581,0.274003,0.016864,0.361071,0.925268,0.006218,all groups,design_7


## 4. Leave one out

In [10]:
checkptFiles = {}
for trainName, trainSet in trainSets.items():
    allTrainData, allValData = [], []
    print('training on all but '+ trainName)
    for name, data in trainSets.items():
        if name != trainName:
            allTrainData = allTrainData + data
            allValData = allValData + valSets[name]

    gcn = FeaStNet()
    history = gcn.trainModel(allTrainData, allValData, epochs=epochs, batch_size=256, flatten=True, logTrans=False, 
                             ssTrans=True, saveDir=saveDir+'allBut_'+ trainName)

    display(plotHistory(history))

    # test
    print('testing on '+trainName+'\n')
    resultsDict = gcn.testModel(testSets[trainName], level='field')
    resultsDict['Trained on'] = ['all groups but test group']*len(resultsDict['mse'])
    resultsDict['Tested on'] = [trainName]*len(resultsDict['mse'])
    results = pivotDict(resultsDict)
    resultsList.extend(results)
    
    # note checkpoint files for use in transfer learning
    checkptFiles[trainName] = gcn.checkptFile
    
pd.DataFrame(resultsList)

training on all but design_5
epoch: 0   trainLoss: 9.8035e-01   valLoss:1.0836e+00  time: 2.14e+00
epoch: 1   trainLoss: 9.0442e-01   valLoss:1.0909e+00  time: 2.12e+00
loading checkpoint 0


alt.Chart(...)

testing on design_5

training on all but design_6
epoch: 0   trainLoss: 1.0529e+00   valLoss:1.1151e+00  time: 2.10e+00
epoch: 1   trainLoss: 9.3899e-01   valLoss:1.1743e+00  time: 2.00e+00
loading checkpoint 0


alt.Chart(...)

testing on design_6

training on all but design_7
epoch: 0   trainLoss: 1.0250e+00   valLoss:1.1227e+00  time: 2.06e+00
epoch: 1   trainLoss: 9.3107e-01   valLoss:1.1437e+00  time: 1.96e+00
loading checkpoint 0


alt.Chart(...)

testing on design_7



,mse,mae,mre,maxAE,mae/peak,maxAE/peak,relEAtPeak,Trained on,Tested on
0,0.000163,0.011892,0.460831,0.019827,0.725857,1.210207,0.033278,test group,design_5
1,0.000198,0.013116,0.483197,0.023848,0.393213,0.714965,0.525229,test group,design_5
2,0.000288,0.013556,0.378542,0.041126,0.238307,0.722956,0.721755,test group,design_5
3,0.001289,0.031469,0.624846,0.081098,0.324650,0.836656,0.836643,test group,design_5
4,0.000174,0.012435,0.481871,0.019110,0.876238,1.346615,0.116389,test group,design_5
...,...,...,...,...,...,...,...,...,...
1615,0.000352,0.015769,0.620358,0.039911,0.496689,1.257102,0.739305,all groups but test group,design_7
1616,0.000033,0.004852,0.252787,0.011612,0.274634,0.657303,0.531057,all groups but test group,design_7
1617,0.000043,0.005320,0.252862,0.015072,0.265117,0.751161,0.583721,all groups but test group,design_7
1618,0.000025,0.004128,0.202716,0.010362,0.226467,0.568544,0.546033,all groups but test group,design_7


## 5. Transfer learning

In [14]:
# restarting with a checkpoint file correctly reassigns self within trainModel(), but the original gcn instance 
# is missing the atributes and does not seem to be modified


allIds = list(range(len(trainSet)))
_, trainIds = train_test_split(allIds, test_size=0.25, shuffle=True, random_state=1234) # train on only 25%

for trainName, trainSet in trainSets.items():
    # load pre-trained model and train on new data
    print('transfer learning on '+trainName)
    gcn = FeaStNet()
    smallTrainSet = [trainSet[i] for i in trainIds]
    history = gcn.trainModel(smallTrainSet, valSets[trainName], restartFile=checkptFiles[trainName], 
                             epochs=epochs, batch_size=256, saveDir=saveDir+'transferLearn_'+ trainName)

    display(plotHistory(history))

#     # test
#     print('testing on '+trainName+'\n')
#     resultsDict = gcn.testModel(testSets[trainName], level='field')
#     resultsDict['Trained on'] = ['transfer learning']*len(resultsDict['mse'])
#     resultsDict['Tested on'] = [trainName]*len(resultsDict['mse'])
#     results = pivotDict(resultsDict)
#     resultsList.extend(results)
        
# pd.DataFrame(resultsList)

transfer learning on design_5
loading restart file
./results/topoTest01/allBut_design_5/checkpoint_0
epoch: 0   trainLoss: 2.3528e+00   valLoss:2.4666e+00  time: 7.42e-01
epoch: 1   trainLoss: 2.3015e+00   valLoss:2.4682e+00  time: 7.69e-01
loading checkpoint 0


alt.Chart(...)

transfer learning on design_6
loading restart file
./results/topoTest01/allBut_design_6/checkpoint_0
epoch: 0   trainLoss: 5.7368e-01   valLoss:7.9449e-01  time: 7.59e-01
epoch: 1   trainLoss: 5.5656e-01   valLoss:8.0019e-01  time: 7.63e-01
loading checkpoint 0


alt.Chart(...)

transfer learning on design_7
loading restart file
./results/topoTest01/allBut_design_7/checkpoint_0
epoch: 0   trainLoss: 6.6102e-01   valLoss:5.8668e-01  time: 7.55e-01
epoch: 1   trainLoss: 6.2688e-01   valLoss:5.8349e-01  time: 7.50e-01
loading checkpoint 1


alt.Chart(...)

In [15]:
gcn.ssTrans

AttributeError: 'FeaStNet' object has no attribute 'ssTrans'

## 6. Plot results

In [ ]:
df = pd.DataFrame(resultsList)
order = ['test group', 'all groups']
barChart = alt.Chart(df).mark_bar().encode(
    x=alt.X('Trained on:N', sort=order, title='', axis=alt.Axis(ticks=False, labels=False)),
    y=alt.Y('mean(mse):Q', scale=alt.Scale(type='log'), axis=alt.Axis(tickCount=8, format=".0e"), title='MSE'),
    color=alt.Color('Trained on:N', sort=order),
    opacity = alt.OpacityValue(0.8),
    tooltip='mean(mse):Q'
).properties(width=75, height=200)

scatter = alt.Chart(df).mark_circle(size=20).encode(
    x=alt.X('Trained on:N', title='', sort=order),
    y=alt.Y('mse:Q', scale=alt.Scale(type='log')),
    color=alt.Color('Trained on:N', sort=order),
    opacity = alt.OpacityValue(0.3),
    tooltip='mse:Q'
)

alt.layer(barChart, scatter, data=df).facet(
    column=alt.Column('Tested on:N'))